<a href="https://colab.research.google.com/github/yanann11/nebius_llm_course/blob/main/topic1/1.7_creating_an_llm-powered_character_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMOps Essentials [Solutions] 1.7. Creating an LLM-powered character.ipynb

# Setting up



In [ ]:
!pip install -q openai

In [ ]:
import os

with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key

In [ ]:
from collections import defaultdict, deque
from openai import OpenAI
from typing import Dict, Any, Optional
import datetime
import string
import random
from dataclasses import dataclass

@dataclass
class NPCConfig:
    world_description: str
    character_description: str
    history_size: int = 10
    has_scratchpad: bool = False

class NPCFactoryError(Exception):
    """Base exception class for NPC Factory errors."""
    pass

class NPCNotFoundError(NPCFactoryError):
    """Raised when trying to interact with a non-existent NPC."""
    def __init__(self, npc_id: str):
        self.npc_id = npc_id
        super().__init__(f"NPC with ID '{npc_id}' not found")

class SimpleChatNPC:
    def __init__(self, client: OpenAI, model: str, config: NPCConfig):
        self.client = client
        self.model = model
        self.config = config
        self.chat_histories = defaultdict(lambda: deque(maxlen=config.history_size))

    def get_system_message(self) -> Dict[str, str]:
        """Returns the system message that defines the NPC's behavior."""
        character_description = self.config.character_description

        if self.config.has_scratchpad:
            character_description += """
You can use scratchpad for thinking before you answer: whatever you output between #SCRATCHPAD and #ANSWER won't be shown to anyone.
You start your output with #SCRATCHPAD and after you've done thinking, you #ANSWER"""

        return {
            "role": "system",
            "content": f"""WORLD SETTING: {self.config.world_description}
###
{character_description}"""
        }

    def chat(self, user_message: str, user_id: str) -> str:
        """Process a user message and return the NPC's response."""
        messages = [self.get_system_message()]

        # Add conversation history
        history = list(self.chat_histories[user_id])
        if history:
            messages.extend(history)

        # Add new user message
        user_message_dict = {
            "role": "user",
            "content": user_message
        }
        self.chat_histories[user_id].append(user_message_dict)
        messages.append(user_message_dict)

        try:
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=0.6
            )

            response = completion.choices[0].message.content

            # Handle scratchpad if enabled
            response_clean = response
            if self.config.has_scratchpad:
                import re
                scratchpad_match = re.search(r"#SCRATCHPAD(:?)(.*?)#ANSWER(:?)", response, re.DOTALL)
                if scratchpad_match:
                    response_clean = response[scratchpad_match.end():].strip()


            # Store response in history, including the scratchpad
            self.chat_histories[user_id].append({
                "role": "assistant",
                "content": response
            })

            # Return the message to the user without a scratchpad
            return response_clean

        except Exception as e:
            return f"Error: {str(e)}"

class NPCFactory:
    def __init__(self, client: OpenAI, model: str):
        self.client = client
        self.model = model
        self.npcs: Dict[str, SimpleChatNPC] = {}
        self.user_ids: Dict[str, str] = {}  # username -> user_id mapping

    def generate_id(self) -> str:
        """Generate a random unique identifier."""
        return ''.join(random.choice(string.ascii_letters) for _ in range(8))

    def register_user(self, username: str) -> str:
        """Register a new user and return their unique ID.
        If username already exists, appends a numerical suffix."""
        base_username = username
        suffix = 1

        # Keep trying with incremented suffixes until we find an unused name
        while username in self.user_ids:
            username = f"{base_username}_{suffix}"
            suffix += 1

        user_id = self.generate_id()
        self.user_ids[username] = user_id
        return user_id

    def register_npc(self, world_description: str, character_description: str,
                     history_size: int = 10, has_scratchpad: bool = False) -> str:
        """Create and register a new NPC, returning its unique ID."""
        npc_id = self.generate_id()

        config = NPCConfig(
            world_description=world_description,
            character_description=character_description,
            history_size=history_size,
            has_scratchpad=has_scratchpad
        )

        self.npcs[npc_id] = SimpleChatNPC(self.client, self.model, config)
        return npc_id

    def chat_with_npc(self, npc_id: str, user_id: str, message: str) -> str:
        """Send a message to a specific NPC from a specific user.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user
            message: The message to send

        Returns:
            The NPC's response

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        npc = self.npcs[npc_id]
        return npc.chat(message, user_id)

    def get_npc_chat_history(self, npc_id: str, user_id: str) -> list:
        """Retrieve chat history between a specific user and NPC.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user

        Returns:
            List of message dictionaries containing the chat history

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        return list(self.npcs[npc_id].chat_histories[user_id])

# Practice: exploring the NPC factory and making a naive service prototype

## Task 1. Make your own character!

Choose a world and a character, create descriptions, and chat with the NPC you design.

For now, we recommend selecting an existing culture or a well-known fandom, along with an existing person or character. Otherwise, you'll need to provide a lot of context to maintain consistency. (And we'll cover best practices for that in Week 3.) That said, feel free to experiment with anything you like! :) Have fun!

Give your character clear goals. While chatting with the NPC, observe their tone of voice, their assumptions about you, and how effectively they pursue their objectives. Be mindful of hallucinations. Start reflecting on what you'd like to improve and experiment with modifying your prompts. But don't overwork it, for now. Over the next two weeks, we'll learn more tricks for making your characters better!

In [ ]:
# <YOUR EXPERIMENTS HERE>

## Task 2. Creating a chat bot arena

In this task, you'll create a `ChatArena` class that will host conversations between two characters. Having a Chat Arena is a great way of exploring NPC behavior without having to carry on the conversations manually. For example, we'll be using it for character evaluation.

Since NPC Factory works with user-NPC interactions, you'll have to register both NPCs as users inside the arena and call them by the generated `user_id`'s when it's their time to speak up as users.

Also, you'll need to force start the conversation with some initial prompt of the first NPC. It can be `"Hello! Who are you?"` or something else depending on what you decide to evaluate.

Below, you'll find examples showing the target interface of the Arena. Also we provide a skeleton of the class, if that can be of help. And, as always, if you get stuck, don't hesitate to ask LLMs for help! Just be sure to provide both the NPC Factory implementation and the examples below.

**Here's our solution**

In [ ]:
from collections import defaultdict
from typing import Dict, Any, List, Tuple
import datetime
import uuid
from dataclasses import dataclass

@dataclass
class Message:
    sender: str
    content: str
    timestamp: datetime.datetime

class ChatArenaError(Exception):
    """Base exception class for Chat Arena errors."""
    pass

class ConversationNotFoundError(ChatArenaError):
    """Raised when trying to interact with a non-existent conversation."""
    def __init__(self, conversation_id: str):
        self.conversation_id = conversation_id
        super().__init__(f"Conversation with ID '{conversation_id}' not found")

class NPCNotFoundInArenaError(ChatArenaError):
    """Raised when trying to interact with a non-existent NPC in the arena."""
    def __init__(self, npc_id: str):
        self.npc_id = npc_id
        super().__init__(f"NPC with ID '{npc_id}' not found in arena")

class ChatArena:
    def __init__(self, npc_factory):
        """Initialize ChatArena with an NPCFactory instance."""
        self.npc_factory = npc_factory
        self.conversations = defaultdict(lambda: {
            'npc1_id': None,
            'npc2_id': None,
            'messages': [],
            'current_turn': None,
            'metrics': defaultdict(float)
        })
        self.metrics = defaultdict(lambda: defaultdict(float))

        # Map NPC IDs to their corresponding user IDs
        self.npc_user_ids: Dict[str, str] = {}

    def register_npc_in_arena(self, npc_id: str) -> str:
        """Register an NPC as a user in the arena.

        Args:
            npc_id: ID of the NPC to register

        Returns:
            user_id: The user ID assigned to this NPC

        Raises:
            NPCNotFoundInArenaError: If the NPC ID doesn't exist in factory
        """
        if npc_id not in self.npc_factory.npcs:
            raise NPCNotFoundInArenaError(npc_id)

        if npc_id not in self.npc_user_ids:
            # Register NPC as a user and store the mapping
            user_id = self.npc_factory.register_user(f"npc_{npc_id}")
            self.npc_user_ids[npc_id] = user_id

        return self.npc_user_ids[npc_id]

    def start_conversation(self, npc1_id: str, npc2_id: str,
                           initial_prompt: str = "Hello! Who are you?") -> str:
        """Start a conversation between two NPCs.

        Args:
            npc1_id: ID of the first NPC
            npc2_id: ID of the second NPC
            initial_prompt: Optional initial message to start the conversation

        Returns:
            conversation_id: Unique identifier for the conversation

        Raises:
            NPCNotFoundInArenaError: If either NPC ID doesn't exist
        """
        # Register both NPCs if they haven't been registered yet
        npc1_user_id = self.register_npc_in_arena(npc1_id)
        npc2_user_id = self.register_npc_in_arena(npc2_id)

        conversation_id = str(uuid.uuid4())

        # Initialize conversation
        self.conversations[conversation_id].update({
            'npc1_id': npc1_id,
            'npc2_id': npc2_id,
            'npc1_user_id': npc1_user_id,
            'npc2_user_id': npc2_user_id,
            'messages': [],
            'current_turn': npc1_id,
            # Create mappings for easier turn management
            'partner_npc': {
                npc1_id: npc2_id,
                npc2_id: npc1_id
            },
            'recipient_user': {
                npc1_id: npc2_user_id,
                npc2_id: npc1_user_id
            }
        })

        # Add initial system message if provided
        if initial_prompt:
            self._add_message(conversation_id, 'system', initial_prompt)

        return conversation_id

    def _add_message(self, conversation_id: str, sender: str, content: str) -> None:
        """Add a message to the conversation history."""
        message = Message(
            sender=sender,
            content=content,
            timestamp=datetime.datetime.now()
        )
        self.conversations[conversation_id]['messages'].append(message)

    def _get_conversation(self, conversation_id: str) -> Dict[str, Any]:
        """Retrieve the conversation object or raise an error if not found."""
        if conversation_id not in self.conversations:
            raise ConversationNotFoundError(conversation_id)
        return self.conversations[conversation_id]

    def _get_prompt(self, conv: Dict[str, Any]) -> str:
        """Determine the prompt from the latest message, or use a default."""
        messages = conv['messages']
        if messages:
            return messages[-1].content
        return "Hello! Who are you?"

    def _switch_turn(self, conv: Dict[str, Any]) -> None:
        """Switch the turn to the partner NPC."""
        current_npc = conv['current_turn']
        conv['current_turn'] = conv['partner_npc'][current_npc]

    def run_turn(self, conversation_id: str) -> Tuple[bool, str]:
        """Run a single turn in the conversation.

        Args:
            conversation_id: ID of the conversation to progress

        Returns:
            Tuple of (success: bool, response: str)

        Raises:
            ConversationNotFoundError: If the conversation ID doesn't exist
        """
        try:
            conv = self._get_conversation(conversation_id)
            current_npc_id = conv['current_turn']
            prompt = self._get_prompt(conv)
            recipient_user_id = conv['recipient_user'][current_npc_id]

            # Get response from the current NPC
            response = self.npc_factory.chat_with_npc(
                npc_id=current_npc_id,
                user_id=recipient_user_id,
                message=prompt
            )

            # Add response to conversation history
            self._add_message(conversation_id, current_npc_id, response)

            # Switch to the partner NPC for the next turn
            self._switch_turn(conv)

            return True, response

        except Exception as e:
            return False, f"Error during turn: {str(e)}"

    def run_conversation(self, conversation_id: str, max_turns: int = 10,
                         verbose: bool = False) -> List[Message]:
        """Run a conversation for a specified number of turns."""
        self._get_conversation(conversation_id)  # Will raise error if not found

        turns_completed = 0
        while turns_completed < max_turns:
            success, response = self.run_turn(conversation_id)
            if not success:
                break

            if verbose:
                current_npc = int(turns_completed % 2) + 1
                print(f"**NPC {current_npc}, turn {turns_completed + 1}**: {response}\n\n")

            turns_completed += 1

        return self.conversations[conversation_id]['messages']

    def evaluate_conversation(self, conversation_id: str) -> Dict[str, float]:
        """Evaluate a conversation and return metrics."""
        conv = self._get_conversation(conversation_id)
        messages = conv['messages']

        metrics = defaultdict(float)

        # Exclude system messages
        npc_messages = [m for m in messages if m.sender != 'system']

        # Basic metrics
        metrics['turn_count'] = len(npc_messages)
        metrics['avg_response_length'] = (
            sum(len(m.content) for m in npc_messages) / len(npc_messages)
            if npc_messages else 0
        )

        # Calculate average response time
        if len(npc_messages) > 1:
            response_times = []
            for i in range(1, len(npc_messages)):
                time_diff = npc_messages[i].timestamp - npc_messages[i-1].timestamp
                response_times.append(time_diff.total_seconds())
            metrics['avg_response_time'] = sum(response_times) / len(response_times)
        else:
            metrics['avg_response_time'] = 0.0

        return metrics

    def get_conversation_history(self, conversation_id: str) -> List[Message]:
        """Get the full history of a conversation."""
        conv = self._get_conversation(conversation_id)
        return conv['messages']


This is how we are going to use the class:

In [ ]:
# Initialize the system
client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

model = "meta-llama/Meta-Llama-3.1-405B-Instruct"

npc_factory = NPCFactory(client=client, model=model)
npc_arena = ChatArena(npc_factory)

# Create two NPCs
knight_id = npc_factory.register_npc(
    world_description="Medieval London, XIII century",
    character_description="A proud knight at Edward I's court",
    has_scratchpad=True
)

merchant_id = npc_factory.register_npc(
    world_description="Medieval London, XIII century",
    character_description="A wealthy merchant from the Hanseatic League",
    has_scratchpad=True
)

# Start a conversation between them
conv_id = npc_arena.start_conversation(knight_id, merchant_id)

# Run the conversation
messages = npc_arena.run_conversation(conv_id, max_turns=6, verbose=True)

**NPC 1, turn 1**: Greetings good fellow! I am Sir Edward de Montfort, a humble knight in the service of our great King Edward I. 'Tis an honor to serve at his court, and I am proud to uphold the noble traditions of chivalry. Pray tell, who are you and what brings you to this fair city of London?


**NPC 2, turn 2**: Greetings Sir Edward de Montfort, 'tis an honor to make your acquaintance. I am Hermann von Stade, a humble merchant from the Hanseatic League. I have come to London to further the interests of our League and to facilitate trade between our nations. I must say, I am in awe of the great King Edward I's accomplishments, and I am proud to do business in a realm that values commerce and prosperity.

Your noble order and the chivalric traditions you uphold are truly admirable, Sir Edward. As a merchant, I may not wield a sword, but I strive to conduct my business with the same honor and integrity that you and your fellow knights embody.

I hope that my presence in London will c

If we want to see the entire conversation history:

In [ ]:
npc_arena.get_conversation_history(conv_id)

[Message(sender='system', content='Hello! Who are you?', timestamp=datetime.datetime(2025, 2, 13, 17, 55, 15, 993968)),
 Message(sender='iFbXnBbs', content="Greetings good fellow! I am Sir Edward de Montfort, a humble knight in the service of our great King Edward I. 'Tis an honor to serve at his court, and I am proud to uphold the noble traditions of chivalry. Pray tell, who are you and what brings you to this fair city of London?", timestamp=datetime.datetime(2025, 2, 13, 17, 55, 20, 259733)),
 Message(sender='GTgjfVXU', content="Greetings Sir Edward de Montfort, 'tis an honor to make your acquaintance. I am Hermann von Stade, a humble merchant from the Hanseatic League. I have come to London to further the interests of our League and to facilitate trade between our nations. I must say, I am in awe of the great King Edward I's accomplishments, and I am proud to do business in a realm that values commerce and prosperity.\n\nYour noble order and the chivalric traditions you uphold are 

Note that scratchpads are absent from the arena's message history. That's because they aren't returned by the NPC, so that the arena never sees them. If you want to check the scratchpads, you'll need to access them through the `npc_factory.get_npc_chat_history` interface.

In our implementation, you can access them in the following way, where

* `npc_arena.conversations[conv_id]["npc2_id"]` is the in our case the `merchant_id` and
* `npc_arena.conversations[conv_id]["npc1_user_id"]` is the `user_id` assigned to the Knight NPC when they are registered as users by the Arena.

The following code will thus return the dialog between the Knight as a user (so, no scratchpads are shown for him) and the Merchant as an NPC (so his scratchpad is shown).

In [ ]:
npc_factory.get_npc_chat_history(
    npc_id=npc_arena.conversations[conv_id]["npc2_id"],
    user_id=npc_arena.conversations[conv_id]["npc1_user_id"]
    )

[{'role': 'user',
  'content': "Greetings good fellow! I am Sir Edward de Montfort, a humble knight in the service of our great King Edward I. 'Tis an honor to serve at his court, and I am proud to uphold the noble traditions of chivalry. Pray tell, who are you and what brings you to this fair city of London?"},
 {'role': 'assistant',
  'content': "#SCRATCHPAD\nThe Hanseatic League's reputation and influence are built on trade and commerce, so I should emphasize my role as a merchant. However, as a representative of the League, I must also show respect for the nobility and the monarch. I should be cautious in my words, as the relationship between the League and the English crown can be complex.\n\nConsidering Sir Edward's background as a knight in the service of King Edward I, I should acknowledge his position and express my admiration for the monarch's accomplishments. A bit of flattery can go a long way in establishing a positive rapport.\n\nAs for my business in London, I can mentio

We also suggest implementing a dummy `evaluate conversation` method. Right now we won't have much exciting here: average answer length in words or characters, or maybe average time answer time. But in the next weeks we'll have more.

In [ ]:
npc_arena.evaluate_conversation(conv_id)

defaultdict(float,
            {'turn_count': 6,
             'avg_response_length': 1252.5,
             'avg_response_time': 17.090555799999997})

Now, play some interactions of your own. Bring together characters with opposite beliefs and goals. Observe what happens.